In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
path_to_file = "/content/hafez.txt"

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read()
text = text.decode(encoding='utf-8')
print ('Total number of characters in the corpus is:', len(text))
print('The first 100 characters of the corpus are as follows:\n', text[:100])

Total number of characters in the corpus is: 1115394
The first 100 characters of the corpus are as follows:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
# The unique characters in the corpus
vocab = sorted(set(text))
print ('The number of unique characters in the corpus is', len(vocab))
print('A slice of the unique characters set:\n', vocab[:10])

The number of unique characters in the corpus is 65
A slice of the unique characters set:
 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']


In [ ]:
# Create a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
# Make a copy of the unique set elements in NumPy array format for later use in the decoding the predictions
idx2char = np.array(vocab)
# Vectorize the text with a for loop
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
# for i in char_dataset.take(5): 
#   print(i.numpy())
seq_length = 100 # The max. length for single input
# examples_per_epoch = len(text)//(seq_length+1) # double-slash for “floor” division
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) 
# for item in sequences.take(5): 
#   print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
BUFFER_SIZE = 10000 # TF shuffles the data only within buffers

BATCH_SIZE = 64 # Batch size

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the traning

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# example_batch_loss  = loss(target_example_batch, example_batch_predictions)
# print("Prediction shape: ", example_batch_predictions.shape, " (batch_size, sequence_length, vocab_size)")
# print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 17s 53ms/step - loss: 2.6728
Epoch 2/30
172/172 [==============================] - 10s 51ms/step - loss: 1.9729
Epoch 3/30
172/172 [==============================] - 10s 51ms/step - loss: 1.7018
Epoch 4/30
172/172 [==============================] - 10s 52ms/step - loss: 1.5498
Epoch 5/30
172/172 [==============================] - 10s 53ms/step - loss: 1.4609
Epoch 6/30
172/172 [==============================] - 10s 54ms/step - loss: 1.3987
Epoch 7/30
172/172 [==============================] - 10s 54ms/step - loss: 1.3522
Epoch 8/30
172/172 [==============================] - 11s 54ms/step - loss: 1.3138
Epoch 9/30
172/172 [==============================] - 11s 55ms/step - loss: 1.2791
Epoch 10/30
172/172 [==============================] - 11s 56ms/step - loss: 1.2449
Epoch 11/30
172/172 [==============================] - 11s 56ms/step - loss: 1.2134
Epoch 12/30
172/172 [==============================] - 11s 57ms/step - loss: 1.1815
E

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            12544     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 49)             50225     
                                                                 
Total params: 4,001,073
Trainable params: 4,001,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, num_generate, temperature, start_string):
  input_eval = [char2idx[s] for s in start_string] # string to numbers (vectorizing)
  input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
  text_generated = [] # Empty string to store our results
  model.reset_states() # Clears the hidden states in the RNN

  for i in range(num_generate): #Run a loop for number of characters to generate
    predictions = model(input_eval) # prediction for single character
    predictions = tf.squeeze(predictions, 0) # remove the batch dimension

    # using a categorical distribution to predict the character returned by the model
    # higher temperature increases the probability of selecting a less likely character
    # lower --> more predictable
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # The predicted character as the next input to the model
    # along with the previous hidden state
    # So the model makes the next prediction based on the previous character
    input_eval = tf.expand_dims([predicted_id], 0) 
    # Also devectorize the number and add to the generated text
    text_generated.append(idx2char[predicted_id]) 

  return (start_string + ''.join(text_generated))

In [ ]:
generated_text = generate_text(
                    model, 
                    num_generate=1000, 
                    temperature=1, 
                    start_string=u"عشق")
print(generated_text)

عشق آينه کرد
کنار آب روی شود ز بهمان غزل خوان بودی


غزل    ۴۳۵

ببين که رقص کنان می‌شکد دل من
که گشت هوای ميکده يک سال من آگاه که خزينه

بدوش معشوق وجودنا اندر آن جا فکن
به اميد سيه اميدم که صبت مقرم ندود زهی سر ببين

اگر چه مردم تو شکه صارم ور نی
به هيچ بابند حريفان نه کنون که خطر زمانه

من مراد آنم که چه گذر از هر کو چشم بر نم
آزم جز لطف و خنيار می‌آورد
دلبر برفت حافظ از اين خرقه پشا نيست
دولت‌ها شد کنم چون لطف خدادان چه کنم

برخانه بدنيم و مکن نرود از کوی
تا در ميان ميان و دريغ مدار

و ادب باشمم و يار بهاران است که خوش باد
که ريز است غم هجران تو بست
يع بر کوی تو از هر فتنه‌اش آن جا
منزل نيست فرح بدان می‌خورم

صوار باد گل چو يار حور و پر از عيب کند
که الاقی من و او نشان فدا می‌کردم

کشته غمزه تو امشاد در دلايت دارم
که ز تيره آبروی خود افکنی به کس آور جلوه گرد نبخشد
دل شود از لعل نوشينان نخواهد شد

مروم نه دل برد و کمرخشاند
ترسم طلب بگذشت در ره عقل می‌دهد
يک نفس بلبل چو از کرم صارا آن خاک پيشان
باشد که خاک معتاقه دلکش عشاق سوخت
جنه‌ای خرج آورد فرياد است خدا را
که به روی شاهودن
که گوش